In [63]:
import sys
import csv
import os.path
import numpy as np

def load_data(csvfile):
	#open file / create headers(column name) and data arrays
	if not os.path.isfile(csvfile):
		exit_error('can\'t find the file ' + csvfile)
	data = []
	with open(csvfile) as csv_iterator:
		data_reader = csv.reader(csv_iterator, delimiter=',')
		for row in data_reader:
			data.append(row)
	csv_iterator.close()
	if len(data) < 1:
		exit_error('file ' + csvfile + ' is empty')
	return data

def f(i):
	if i[1] == 'M':
		return 1
	else:
		return 0

dataRaw = np.array(load_data('./data/data.csv'))
data = []
y = np.array([f(i) for i in dataRaw])
dataRaw = np.delete(dataRaw, [0,1,4,5], 1)
dataRaw = dataRaw.astype('float')
data = np.zeros((dataRaw.shape[1], dataRaw.shape[0]))
for index, feature in enumerate(dataRaw.T):
	data[index] = [(x - min(feature)) / (max(feature) - min(feature)) for x in feature]
x_train = data.T

In [66]:
x_train[range(2)]
print(range(2))

range(0, 2)
